# Agents
There are many types of agent that work in slightly different ways, but the main concepts behind an agent are:

1. The agent has access to tools. Tools include instructions on how to use them and what they are useful for. A tool can be anything that could be used by using text.
2. The agent has a task, usually this is just the prompt from the user.
3. The agent has access to an LLM and a system prompt that tells it how to go about achieving it's task given it's tools
4. The agent will attempt to breakdown its task according to its programming and iteratively take steps to achieve it. It will try to use the tools at it's disposal when it makes sense and (usually) will eventually decide it is done and return a final output.

In [26]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from pandasai import SmartDataframe
import pandas as pd
from langchain.tools import BaseTool, StructuredTool, Tool, tool

from pydantic import BaseModel, Field


class FaultTolerantSmartDataframe:
    def __init__(self, df, config):
        self.sdf = SmartDataframe(df, config=config)
    
    def __call__(self, prompt):
        try:
            return self.sdf.chat(prompt)
        except Exception as e:
            return f'Error: {e}'
        

llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0,
)
df = pd.read_csv('../extra_resources/candy-data.csv')
sdf = FaultTolerantSmartDataframe(df, config={"llm": llm})

    
candy_data_tool = Tool.from_function(
    func=sdf,
    name="Smart Candy Dataset",
    description="Useful to get general information about candies, their classification, price, price and rating. You can ask it questions in natural language and it will answer back.",
    # args_schema=CodeSuggestionsInput
)



In [27]:

from langchain.agents import initialize_agent

model = ChatOpenAI(temperature=0)
tools = [candy_data_tool, PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [28]:
agent.run('What is the minimum combination of candies that together include all categories? Remember there are many candies that can cover multiple categories. Show me evidence this is indeed the minimum set.')



> Entering new  chain...
First, I need to know what the categories are. Then, I can find candies that cover multiple categories and try to find the minimum set.
Action: Smart Candy Dataset
Action Input: What are the candy categories?
Observation: chocolate, fruity, caramel, peanutyalmondy, nougat, crispedricewafer, hard, bar, pluribus
Thought:Now that I know the categories, I need to find candies that cover multiple categories. I'll start by finding candies that cover the most categories.
Action: Smart Candy Dataset
Action Input: Which candies cover the most categories?
Observation: The candies that cover the most categories are: Baby Ruth, Snickers, Snickers Crisper.
Thought:Now I know which candies cover the most categories. I need to check which categories are covered by these candies.
Action: Smart Candy Dataset
Action Input: Which categories are covered by Baby Ruth, Snickers, and Snickers Crisper?
Observation:       competitorname  chocolate  fruity  caramel  peanutyalmondy  no

'The minimum combination of candies that together include all categories are Snickers Crisper and any one of the candies that cover the remaining categories: Gobstopper, Jawbusters, Nerds, Pop Rocks, Runts, Smarties candy, or Strawberry bon bons.'